In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

In [2]:
def lyap_spectrum_QR(Js,T):
    K,n = Js.shape[0],Js.shape[-1]    
    Q,R = np.linalg.qr(Js[0])
    lyaps = np.zeros((n,))

    for t in range(1,K):
        Q = Js[t] @ Q
        

        # Need diagonal of R to be positive, so rewrite Q = Q @ S and R = S @ R, where S_ii = sign(R_ii)
        S = np.diag(np.sign(np.diag(R)))
        lyaps += np.log(np.diag(S @ R))
        
        Q = Q @ S

        #add diagonal of R to running lyapunov spectrum estimate
        
        Q,R = np.linalg.qr(Q)

    lyaps /= T
    
    return lyaps
    
    

In [3]:
def lorenz(w,t, sigma, rho, beta):
    
    x,y,z = w[0],w[1],w[2]

    x_dot = sigma*(y - x)
    y_dot = x*(rho-z) - y
    z_dot = x*y - beta*z
    
    w_dot = [x_dot,y_dot,z_dot] 
    

    
    return w_dot



def get_lorenz_jacobian(sol,dt,sigma = 10, rho = 28,beta = 8/3):
    num_steps = sol.shape[0]
    Js = np.zeros((num_steps,3,3))
    I = np.eye(3)
    for t in range(num_steps):        
        Js[t,0,:] = [-sigma,sigma,0]
        Js[t,1,:] = [-sol[t,2]+ rho,-1,-sol[t,0]]
        Js[t,2,:] = [sol[t,1],sol[t,0],-beta]
        
        Js[t] = I + dt*Js[t]
        
    return Js



In [4]:
w0 = [.01,.01,.01]
T = 100
dt = .001
K = int(T/dt)
t = np.linspace(0, T, K)
sol = odeint(lorenz, w0, t, args=(10, 28, 8/3))

Js = get_lorenz_jacobian(sol,dt)
lyaps = lyap_spectrum_QR(Js,T)
print(lyaps)

[  0.84380306  -0.01622436 -14.52583331]
